make a gene table with:
contig, contig_length, gene_name, gene_start, gene_stop, gene_orient, Evidence_source, evidence_acession, evidence_description, virus_score(or category)

In [1]:
import os
import re
import sys
import pandas as pd
import glob
import numpy as np

import itertools
from itertools import tee
import csv

import statistics
from statistics import mean
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import math
import collections
import bisect


In [2]:
repeat_table = "/Users/u241374/mike_tisza/sandbox/test_ct2_0802a/ct2_tmp/hallmark_contigs_terminal_repeat_summary.tsv"

phan_tab_directory = "/Users/u241374/mike_tisza/sandbox/test_ct2_0802a/ct2_tmp/reORF/phan_split"

prod_tab_directory = "/Users/u241374/mike_tisza/sandbox/test_ct2_0802a/ct2_tmp/reORF/prod_split"

first_pyhmmer_table = "/Users/u241374/mike_tisza/sandbox/test_ct2_0802a/ct2_tmp/reORF_pyhmmer/pyhmmer_report_AAs.tsv"

second_pyhmmer_table = "/Users/u241374/mike_tisza/sandbox/test_ct2_0802a/ct2_tmp/second_reORF_pyhmmer/pyhmmer_report_AAs.tsv"

mmseqs_CDD_table = "/Users/u241374/mike_tisza/sandbox/test_ct2_0802a/ct2_tmp/third_reORF_combined/summary_no2_AAs_vs_CDD.besthit.tsv"

viral_cdds_list = "/Users/u241374/mike_tisza/cmmr_repos/Cenote-Taker2/viral_cdds_and_pfams_191028.txt"

out_dir = "/Users/u241374/mike_tisza/sandbox/figures3"

if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

In [3]:
try:

    phan_files = glob.glob(os.path.join(phan_tab_directory, "*.bed"))

    df_from_each_phan = (pd.read_csv(phan, sep = "\t", header = None,
                                    names = ["contig", "gene_start", "gene_stop", "gene_name", 
                                            "gene_score", "gene_orient"])
                                    for phan in phan_files)
    phan_gene_df = pd.concat(df_from_each_phan, ignore_index=True)

    phan_gene_df = phan_gene_df.drop("gene_score", axis = 1)

except:
    print("no phanotate tables")
    phan_gene_df = pd.DataFrame()



In [4]:
try:

    prod_files = glob.glob(os.path.join(prod_tab_directory, "*.gff"))

    df_from_each_prod = (pd.read_csv(prod, sep = "\t", header = None, comment='#',
                                    names = ["contig", "gene_caller", "feature_type", "gene_start", 
                                            "gene_stop", "score", "gene_orient", "frame", "attribute"])
                                    for prod in prod_files)
    prod_gene_df = pd.concat(df_from_each_prod, ignore_index=True)

    prod_gene_df = prod_gene_df[["contig", "gene_start", "gene_stop", "attribute", "gene_orient"]]

    prod_gene_df["semi_pos"] = prod_gene_df["attribute"].str.find(";")
    prod_gene_df["gene_IDstr"] = prod_gene_df.apply(
        lambda x: x["attribute"][0:x["semi_pos"]], axis = 1)
    
    prod_gene_df["under_pos"] = prod_gene_df["gene_IDstr"].str.rfind("_")
    prod_gene_df["gene_num"] = prod_gene_df.apply(
        lambda x: x["gene_IDstr"][x["under_pos"]:len(x["gene_IDstr"])], axis = 1)
    
    prod_gene_df["gene_name"] = prod_gene_df["contig"] + prod_gene_df["gene_num"]


except:
    print("no prodigal tables")
    prod_gene_df = pd.DataFrame()

prod_gene_df.head()

,contig,gene_start,gene_stop,attribute,gene_orient,semi_pos,gene_IDstr,under_pos,gene_num,gene_name
0,test_ct2_0802a3,2,334,ID=1_1;partial=10;start_type=ATG;rbs_motif=Non...,-,6,ID=1_1,4,_1,test_ct2_0802a3_1
1,test_ct2_0802a3,339,959,ID=1_2;partial=00;start_type=ATG;rbs_motif=Non...,+,6,ID=1_2,4,_2,test_ct2_0802a3_2
2,test_ct2_0802a3,964,1257,ID=1_3;partial=00;start_type=ATG;rbs_motif=GGA...,+,6,ID=1_3,4,_3,test_ct2_0802a3_3
3,test_ct2_0802a3,1295,1774,ID=1_4;partial=00;start_type=ATG;rbs_motif=Non...,-,6,ID=1_4,4,_4,test_ct2_0802a3_4
4,test_ct2_0802a3,2249,3514,ID=1_5;partial=00;start_type=ATG;rbs_motif=Non...,-,6,ID=1_5,4,_5,test_ct2_0802a3_5


In [5]:
if not phan_gene_df.empty and not prod_gene_df.empty:
    print("both")
    both_list = [phan_gene_df, prod_gene_df]
    just_gene_df = pd.concat(both_list, ignore_index=True)
elif not phan_gene_df.empty:
    print("phan")
    just_gene_df = phan_gene_df
elif not prod_gene_df.empty:
    print("prod")
    just_gene_df = prod_gene_df


both


In [6]:
try:
    length_df = pd.read_csv(repeat_table, sep = "\t")[['contig', 'out_length_contig', 'dtr_seq']]

    length_df = length_df.rename(columns={"out_length_contig": "contig_length"})
except:
    print("nope")

In [7]:
try:
    basal_df = just_gene_df.merge(length_df, on = "contig", how = "left")
except:
    print("nope")

In [ ]:
first_pyh_df = pd.read_csv(first_pyhmmer_table, sep = "\t")[['ORFquery', 'target']]

first_pyh_df.head()

In [10]:
# gene_name, Evidence_source, evidence_acession, evidence_description, virus_score(or category)

try:
    first_pyh_df = pd.read_csv(first_pyhmmer_table, sep = "\t")[['ORFquery', 'target']]



    first_pyh_df["gene_name"] = first_pyh_df["ORFquery"]

    first_pyh_df["slash_pos"] = first_pyh_df["target"].str.find("/")
    first_pyh_df["fdash_pos"] = first_pyh_df["target"].str.find("-")


    first_pyh_df["evidence_acession"] = first_pyh_df.apply(
        lambda x: x["target"][x["slash_pos"]+1:x["fdash_pos"]], axis = 1)

    first_pyh_df["evidence_description"] = first_pyh_df.apply(lambda x: x["target"][x["fdash_pos"]+1:], axis = 1)

    first_pyh_df = first_pyh_df[['gene_name', 'evidence_acession', 'evidence_description']]

    first_pyh_df['Evidence_source'] = 'hallmark_hmm'

    first_pyh_df['vscore_category'] = 'common_virus'

except:
    print("nope")



In [11]:
try:
    second_pyh_df = pd.read_csv(second_pyhmmer_table, sep = "\t")[['ORFquery', 'target']]


    second_pyh_df["lpar_pos"] = second_pyh_df["ORFquery"].str.find("(")

    second_pyh_df["gene_name"] = second_pyh_df.apply(lambda x: x["ORFquery"][0:x["lpar_pos"]], axis = 1)

    second_pyh_df["slash_pos"] = second_pyh_df["target"].str.find("/")
    second_pyh_df["fdash_pos"] = second_pyh_df["target"].str.find("-")


    second_pyh_df["evidence_acession"] = second_pyh_df.apply(
        lambda x: x["target"][x["slash_pos"]+1:x["fdash_pos"]], axis = 1)

    second_pyh_df["evidence_description"] = second_pyh_df.apply(lambda x: x["target"][x["fdash_pos"]+1:], axis = 1)

    second_pyh_df = second_pyh_df[['gene_name', 'evidence_acession', 'evidence_description']]

    second_pyh_df['Evidence_source'] = 'common_virus_hmm'

    second_pyh_df['vscore_category'] = 'common_virus'

except:
    print("nope")

In [17]:
cdd_df = pd.read_csv(mmseqs_CDD_table, sep = "\t")[['query', 'target', 'description']]

cdd_df['lpar_pos'] = np.where(cdd_df['query']
                               .str.contains("\("), 
                               cdd_df["query"].str.find("("), len(cdd_df['query']))

#cdd_df["lpar_pos"] = cdd_df["query"].str.find("(")

cdd_df["gene_name"] = cdd_df.apply(lambda x: x["query"][0:x["lpar_pos"]], axis = 1)

cdd_df = cdd_df[['gene_name', 'target', 'description']]

cdd_df = cdd_df.rename(columns={"target": "evidence_acession", "description" : "evidence_description"})

cdd_df['Evidence_source'] = 'mmseqs_cdd'

#cdd_df['vscore_category'] = 'nonviral_gene'

#cdd_df['vscore_category'] = np.where(cdd_df['target']
#                               .str.contains(
#                                   "PHA0",
#                                   case = False), 'common_virus', cdd_df['vscore_category'])




cdd_df.head()

,gene_name,evidence_acession,evidence_description,Evidence_source
0,ct2_b_caccae_b1_10,COG2239,Mg/Co/Ni transporter MgtE (contains CBS domain...,mmseqs_cdd
1,ct2_b_caccae_b1_1001,pfam09561,HpaII restriction endonuclease. This family in...,mmseqs_cdd
2,ct2_b_caccae_b1_1002,cd02136,nitroreductase similar to Mycobacterium smegma...,mmseqs_cdd
3,ct2_b_caccae_b1_1003,TIGR00461,glycine dehydrogenase (decarboxylating). This ...,mmseqs_cdd
4,ct2_b_caccae_b,cd06262,mainly hydrolytic enzymes and related proteins...,mmseqs_cdd


In [ ]:
virlist_df = pd.read_csv(viral_cdds_list, header = None, names = ["evidence_acession"])

virlist_df['vscore_category'] = 'common_virus'

In [ ]:
comb_cdd_df = cdd_df.merge(virlist_df, on = "evidence_acession", how = "left")

comb_cdd_df['vscore_category'] = np.where(comb_cdd_df['evidence_acession']
                               .str.contains(
                                   "PHA0",
                                   case = False), 'common_virus', comb_cdd_df['vscore_category'])

comb_cdd_df['vscore_category'] = np.where(comb_cdd_df['vscore_category'].isna(), 'nonviral_gene',
                                          comb_cdd_df['vscore_category'])

In [ ]:
gene_ann_list = []

if not first_pyh_df.empty:
    gene_ann_list.append(first_pyh_df)

if not second_pyh_df.empty:
    gene_ann_list.append(second_pyh_df)

if not comb_cdd_df.empty:
    gene_ann_list.append(comb_cdd_df)

try:
    gene_ann_df = pd.concat(gene_ann_list, ignore_index=True)

    contig_gene_df = basal_df.merge(gene_ann_df, on = "gene_name", how = "left")

    contig_gene_df['vscore_category'] = np.where(contig_gene_df['vscore_category'].isna(), 'hypothetical_protein',
                                          contig_gene_df['vscore_category'])
    
    contig_gene_df['evidence_description'] = np.where(contig_gene_df['evidence_description'].isna(), 'hypothetical protein',
                                          contig_gene_df['evidence_description'])

except:
    print("nope")

In [ ]:



grouped_df = contig_gene_df.query("contig_length >= 10000").groupby('contig')

In [ ]:


def prune(name, group, out_dir1):

    ## function takes the name and data from the grouped pandas dataframe
    ## grouped_df
    ## and the output directory

    ####define scoring scheme and important variables
    id_list = ['common_virus','hypothetical_protein','nonviral_gene','intergenic']
    score_list = [10,5,-3,0]

    keys = id_list
    values = score_list
    domain_dictionary = dict(zip(keys,values))

    threshold = 0
    window = 5000
    sliiiiiide_to_the_right = 50

    count=0
    count_start=-sliiiiiide_to_the_right

    contig_length1 = group['contig_length'].agg(pd.Series.mode)
    ####

    ## make the score list based on annotations
    vscore_list = list([0] * int(contig_length1.iloc[0]))

    total_len = int(len(vscore_list))

    for index, row in group.iterrows():
        S = domain_dictionary[row['vscore_category']]
        vscore_list[row['gene_start']:row['gene_stop']] = [S] * (row['gene_stop'] - row['gene_start'])

    blocks = int(((len(vscore_list) - window) / sliiiiiide_to_the_right) + 1)
    blocks_2 = blocks + 2 #need this otherwise the last (incomplete/little) block will be cut off!

    dat_list = []
    for i in range(0, blocks_2 * sliiiiiide_to_the_right, sliiiiiide_to_the_right):
        score_result = sum(vscore_list[i:i+window])
        new_let_list = vscore_list[i:i+window]

        if score_result >= 0 :
            PF_result = "pass"
        else:
            PF_result = "fail"

        #counts for later
        VirusGene = new_let_list.count(10)
        HypotheticalGene = new_let_list.count(5)
        BacterialGene = new_let_list.count(-3)
        Intergenic = new_let_list.count(0)

        #vars for count columns
        count = count +1
        count_start += sliiiiiide_to_the_right #same as c_s = c_s + siiii...
        count_stop = count_start+window

        dat_list.append([count, count_start, count_stop, PF_result, score_result, VirusGene,
                          HypotheticalGene, BacterialGene, Intergenic])


    df_0 = pd.DataFrame(dat_list, columns=['Window', 'Position start', 'Position stop',
                                             'Pass/Fail', 'Score', 'VirusGene', 'HypotheticalGene', 
                                             'Intergenic', 'BacterialGene'])

    #Now let's make the smoothed plot

    x = df_0['Window']
    y = np.array(df_0['Score'])
    l = df_0['Window'].count()
    df_empty = pd.DataFrame(index=range(l),columns=range(1))
    for col in df_empty.columns:
        df_empty[col].values[:] = 0

    zero=df_empty[0]

    def smooth(y, box_pts):
        box = np.ones(box_pts)/box_pts
        y_smooth = np.convolve(y, box, mode='same')
        return y_smooth
    smooth_val = 100 #####we can change this if we want!


    #statement for handling short sequences (error called if len(y) < smoothing value)
    if len(y) <= smooth_val:
        smooth_val = int(0.5 * len(y))
    else:
        smooth_val = int(smooth_val)

    smoth = smooth(y,smooth_val)
    idx = np.argwhere(np.diff(np.sign(zero - smoth))).flatten()
    df_val = pd.DataFrame(zero[idx])
    df_val = df_val.reset_index()
    #we will save to figures, but first we need to do the validation steps

    #This is for validating if region is + or -
    df_val.loc[-1] = 1  # adding a row for first position
    df_val.index = df_val.index + 1  # shifting index
    df_val = df_val.sort_index()

    #last position as last row
    df_val.sort_values(by=['index']) #need to sort first otherwise +1 belwo will break things
    new_list = pd.DataFrame(df_val['index'] + 1) #df['index'][:-1] + 1 #add +1 to all for next position is +/-, except for last position, will throw erre - so it deletes it, we'll add it in later

    new_list_2 = new_list['index']

    new_y_val = list(smoth[new_list])   #find position y on smooth line

    #assigning pos / neg for that +1 position
    pos_neg_results = []
    for i in new_y_val:
        if i > 0:
            result = '+'
        else:
            result = '-'
        pos_neg_results.append(result)

    #creating dataframe for next steps
    df_val.drop(df_val.columns[len(df_val.columns)-1], axis=1, inplace=True) #to delete last column, unnamed so tricky to get rid of (?) this does it tho
    df_val['+/- to the right'] = pos_neg_results

    #append +/- and start stop coords from original table
    df_val.rename(columns={'index': 'Window'}, inplace=True)

    df_val['Window']=df_val['Window'].astype(int)
    df_0['Window']=df_0['Window'].astype(int)

    ## merging
    merged_df = df_val.merge(df_0, how = 'inner', on = ['Window'])

    merged_df = merged_df.drop(['Pass/Fail','Score','VirusGene','HypotheticalGene','Intergenic','BacterialGene'], axis = 1)
    merged_df['Chunk_end'] = 'none'
    merged_df['Window midpoint'] = merged_df.iloc[:,[2,3]].median(axis=1)
    merged_df['Window midpoint'] = merged_df['Window midpoint'].astype(int)

    #df edits to accomodate this:
    #we are duplicating the last row of the df to handle a trailing + chunk (w/ no y=0 intercept to close the chunk)
    #merged_df = merged_df.append(merged_df[-1:])
    merged_df = pd.concat([merged_df, merged_df[-1:]])
    #now need to make it read actual last stop position (this os not rounded per window like the other coords)
    merged_df = merged_df.replace(merged_df.iloc[-1][3],(total_len+1))

    #now let's get the coordinates for the > 0 'chunks'
    #iterate over for true hit testing
    def pairwise(iterable):
        "s -> (s0,s1), (s1,s2), (s2, s3), ..."
        a, b = tee(iterable)
        next(b, None)
        return zip(a, b)

    #this is to define the chunks, accounting for all the ways the graph can look
    #note: leading and trailing here mean a chunk at the start or end of the graph that

    ## use nearby ORF boundaries to refine cutoffs:

    def right_cutoff(position, groupframe):
        calced_end =  int(position)
        rbisect_pos = bisect.bisect(list(groupframe['gene_stop']), calced_end)
        rchunk_cutoff = list(groupframe['gene_stop'])[rbisect_pos-1]
        return rchunk_cutoff

    def left_cutoff(position, groupframe):
        calced_start =  int(position)
        lbisect_pos = bisect.bisect(list(groupframe['gene_start']), calced_start)
        lchunk_cutoff = list(groupframe['gene_stop'])[lbisect_pos]
        return lchunk_cutoff

    ddf_list = []

    for (i1, row1), (i2, row2) in pairwise(merged_df.iterrows()):
        #for a leading chunk
        if row1['+/- to the right'] == '+' and \
            row1["Position start"] == 0 and \
            row1["Position stop"] != (total_len + 1):

                ddf = ["Chunk_" + str(i1), 
                       row1["Position start"], 
                       right_cutoff(row2["Window midpoint"], group)]
                ddf_list.append(ddf)
        #for a contained chunk
        if row1['+/- to the right'] == '+' and \
            row1["Position start"] != 0 and \
            row1["Position stop"] != (total_len + 1):

                ddf = ["Chunk_" + str(i1), 
                       left_cutoff(row1["Window midpoint"], group), 
                       right_cutoff(row2["Window midpoint"], group)]
                ddf_list.append(ddf)
        #3. for a trailing chunk
        if row1['+/- to the right'] == '+' and \
            row1["Position start"] != 0 and \
            row1["Position stop"] == (total_len + 1):
                
                ddf = ["Chunk_" + str(i1), 
                       left_cutoff(row1["Window midpoint"], group), 
                       row2["Position stop"]]
                ddf_list.append(ddf)

        #4. for graphs with no leading and no trailing chunk (for graphs with no y = 0 intercept -> this is is
        #a differently-defined statemnt below b/c the empty file gets appended w/ stuff above from older files when
        #it's in the loop, ALSO the criterion gets fulfilled by contained cunks which means duplicate csv rows for chunks (defined diffrently to specifiy the rules)
        if merged_df.iloc[0,1] == '+' and \
            merged_df.iloc[0,2] == 0 and \
            merged_df.iloc[0,3] == (total_len + 1): #if first column last(2nd row) == last -1 then its one chunk
                rep_list = [('Chunk_0', '0', (total_len+1))]
                ddf_list = rep_list
        else:
                ddf_list = ddf_list

    ## make chunk csv
    chunk_df = pd.DataFrame(ddf_list, columns=["chunk_number", "left_cutoff", "right_cutoff"])

    chunk_sum_file = os.path.join(out_dir1, name + ".chunks.tsv")

    chunk_df.to_csv(chunk_sum_file, sep = "\t", index = False)

    ###Find optimal location on plot to place hallmark marker
    vir_bait_table = group[['gene_start', 'gene_stop', 'Evidence_source']]\
        .query("Evidence_source == 'hallmark_hmm'")
    vir_bait_table['mean'] = round(vir_bait_table[['gene_start', 'gene_stop']].mean(axis=1))
    vir_bait_table_med_list = list(vir_bait_table['mean'])

    points_list = []
    for item in vir_bait_table_med_list:
        eq = round(((item - 2500) + 50) / 50)
        if eq >= len(x):
            plot_point = (len(x) - 1) #1 because it can't = len, has to be less
        else:
            plot_point = eq

        points_list.append(plot_point)

    new_points_list = [1 if i <=0 else i for i in points_list]

    df_0['smoothy'] = smooth(df_0['Score'],100)

    #FIGURES
    pdf_outname = os.path.join(out_dir1, name + ".figures.pdf")

    figures = PdfPages(pdf_outname)

    ## first figure, shows smoothed average

    plt.plot(x, y, 'o', ms=0.6)
    plt.axhline(0, 0, l)
    plt.plot(x, df_0['smoothy'], 'c', lw=2)
    ## add halmark gene markers
    plt.plot(x, df_0['smoothy'], 'y', markevery = (new_points_list), ms=11.0, marker = '*')
    plt.title("Viral region calls")
    plt.xlabel('Window')
    plt.ylabel('Score')
    plt.rc('axes', titlesize=6.8)     # fontsize of the axes title
    plt.rc('xtick', labelsize=5)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=5)    # fontsize of the tick labels
    plt.rc('legend', fontsize=5)    # legend fontsize
    plt.rc('figure', titlesize=8)  # fontsize of the figure title
    plt.grid(True)
    idx = np.argwhere(np.diff(np.sign(zero - smooth(y,100)))).flatten()
    plt.plot(x[idx], zero[idx],  'ro', ms=5.0)


    plt.plot()
    plt.savefig(figures, format='pdf')
    plt.close()

    ## second figure shows gene type distribution
    mycol = (["#e7ba52", "#637939", "#7b4173", "#d6616b"])
    df_0[['VirusGene','HypotheticalGene','BacterialGene','Intergenic']].plot(color = mycol)
    plt.grid(True)
    plt.xlabel('Window')
    plt.ylabel('Count')
    plt.title('Character counts')
    plt.rc('axes', titlesize=6.8)     # fontsize of the axes title
    plt.rc('xtick', labelsize=5)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=5)    # fontsize of the tick labels
    plt.rc('legend', fontsize=5)    # legend fontsize
    plt.rc('figure', titlesize=8)  # fontsize of the figure title
    plt.savefig(figures, format='pdf')
    plt.close()

    figures.close()

    


In [ ]:
## test bisect

for name, group in grouped_df:
    group.notnull
    prune(name, group, out_dir)


#also output contig_gene_df to table file